In [ ]:
# default_exp dash

# Dash

> Plotly Dash application to visualize and recommendations

https://plot.ly/dash/

In [ ]:
#hide
from nbdev.export import *

In [ ]:
#export
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

from functools import lru_cache

import pandas as pd
import requests

from word2vec_recommender.core import *
from word2vec_recommender.recommender import KnnRecommender

In [ ]:
#export
movies_df = pd.read_csv("./data/ml-latest-small/movies.csv")
links_df = pd.read_csv("./data/ml-latest-small/links.csv")

In [ ]:
links_df.

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9737,193581,5476944,432131.0
9738,193583,5914996,445030.0
9739,193585,6397426,479308.0
9740,193587,8391976,483455.0


In [ ]:
#export
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, 
    external_stylesheets=external_stylesheets)

app.layout = html.Div(children=[
    html.H1(children='MovieLens Recommender based on Word2Vec'),

    html.Div(children='''
        Dash: A web application framework for Python.
    '''),

    dcc.Dropdown(
        id="movie-id",
        options=movies_df[["title", "movieId"]].apply(lambda r:  {"value": r["movieId"], "label": r["title"]} ,axis=1).tolist(),
    ),

    html.Div(id="selected-movie-id", children=""),

    dcc.Graph(
        id='example-graph',
        figure={
            'data': [
                {'x': [1, 2, 3], 'y': [4, 1, 2], 'type': 'bar', 'name': 'SF'},
                {'x': [1, 2, 3], 'y': [2, 4, 5], 'type': 'bar', 'name': u'Montréal'},
            ],
            'layout': {
                'title': 'Dash Data Visualization'
            }
        }
    )
])


@app.callback(
    Output(component_id='selected-movie-id', component_property='children'),
    [Input(component_id='movie-id', component_property='value')]
)
def update_output_div(input_value):
    return f"Selected movie = {input_value}"

In [ ]:
#export
def start_dash(debug: bool = True):
    app.run_server(debug=debug, port=9998)

In [ ]:
#hide
start_dash(False)

* Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:9998/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Dec/2019 15:36:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Dec/2019 15:36:54] "GET /_dash-component-suites/dash_renderer/react@16.v1_2_2m1576762278.8.6.min.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Dec/2019 15:36:54] "GET /_dash-component-suites/dash_renderer/prop-types@15.v1_2_2m1576762278.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Dec/2019 15:36:54] "GET /_dash-component-suites/dash_renderer/polyfill@7.v1_2_2m1576762278.7.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Dec/2019 15:36:54] "GET /_dash-component-suites/dash_renderer/react-dom@16.v1_2_2m1576762278.8.6.min.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Dec/2019 15:36:54] "GET /_dash-component-suites/dash_html_components/dash_html_components.v1_0_2m1576762280.min.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Dec/2019 15:36:54] "GET /_dash-co